In [1]:
# NOTE: reduced implementation might require slightly different parameters for same results

## Notebook Config

In [1]:
# Imports
import gc
import os
import sys
import h5py
import random
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

In [2]:
# Relative Imports
package_path = Path(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
sys.path.insert(0, str(package_path))

from src.expressive_leaky_memory_neuron_v2 import ELM
from src.expressive_leaky_memory_neuron_forget import ELMf
from src.neuronio.neuronio_data_utils import (
    NEURONIO_DATA_DIM, 
    NEURONIO_LABEL_DIM, 
    get_data_files_from_folder, 
    parse_sim_experiment_file,
    visualize_training_batch,
)
from src.neuronio.neuronio_data_loader_filtered import NeuronIO
from src.neuronio.neuronio_train_utils import NeuronioLoss
from src.neuronio.neuronio_eval_utils import (
    NeuronioEvaluator, 
    compute_test_predictions_multiple_sim_files, 
    filter_and_extract_core_results,
)

In [3]:
# General Config
general_config = dict()
general_config["seed"] = 0
general_config["device"] = 'cuda' if torch.cuda.is_available() else 'cpu'
general_config["short_training_run"] = True # TODO: change to full length run
general_config["verbose"] = True #general_config["short_training_run"]
torch_device = torch.device(general_config["device"])
print("Torch Device: ", torch_device)

Torch Device:  cuda


In [4]:
# Seeding & Determinism
os.environ['PYTHONHASHSEED'] = str(general_config["seed"])
random.seed(general_config["seed"])
np.random.seed(general_config["seed"])
torch.manual_seed(general_config["seed"])
torch.cuda.manual_seed(general_config["seed"])
torch.backends.cudnn.deterministic = True

## Data, Model, Training Config

In [5]:
# NOTE: this step requires you having downloaded the dataset

# Download Train Data: 
# https://www.kaggle.com/datasets/selfishgene/single-neurons-as-deep-nets-nmda-train-data
# Download Test Data: 
# https://www.kaggle.com/datasets/selfishgene/single-neurons-as-deep-nets-nmda-test-data # Data_test

# Location of downloaded folders
data_dir_path = Path("D:/NeuronIO").expanduser().resolve() # TODO: change to neuronio data path
train_data_dir_path = data_dir_path / "train"  # TODO: change to train subfolder
test_data_dir_path = data_dir_path / "test/Data_test"  # TODO: change to test subfolder

In [6]:
# Data Config

data_config = dict()
train_data_dirs = [
    str(train_data_dir_path / "full_ergodic_train_batch_2"),
    str(train_data_dir_path / "full_ergodic_train_batch_3"),
    str(train_data_dir_path / "full_ergodic_train_batch_4"),
    str(train_data_dir_path / "full_ergodic_train_batch_5"),
    str(train_data_dir_path / "full_ergodic_train_batch_6"),
    str(train_data_dir_path / "full_ergodic_train_batch_7"),
    str(train_data_dir_path / "full_ergodic_train_batch_8"),
    str(train_data_dir_path / "full_ergodic_train_batch_9"),
    str(train_data_dir_path / "full_ergodic_train_batch_10"),
]
valid_data_dirs = [str(train_data_dir_path / "full_ergodic_train_batch_1")]
test_data_dirs = [str(test_data_dir_path)]

data_config["train_data_dirs"] = train_data_dirs
data_config["valid_data_dirs"] = valid_data_dirs
data_config["test_data_dirs"] = test_data_dirs

data_config["data_dim"] = NEURONIO_DATA_DIM 
data_config["label_dim"] = NEURONIO_LABEL_DIM

In [7]:
# Model Config

model_config = dict()
model_config["num_input"] = data_config["data_dim"]
model_config["num_output"] = data_config["label_dim"]
model_config["num_memory"] = 10
#model_config["mlp_hidden_size"] = 10
model_config["memory_tau_max"] = 150.0
model_config["num_branch"] = 45
model_config["num_synapse_per_branch"] = 100
model_config["input_to_synapse_routing"] = "neuronio_routing"

In [8]:
# Training Config

train_config = dict()
train_config['forget_gate'] = True
train_config["num_epochs"] = 5 if general_config["short_training_run"] else 30
train_config["learning_rate"] = 5e-4
train_config["batch_size"] = 16 if general_config["short_training_run"] else 8
train_config["batches_per_epoch"] = 1000 if general_config["short_training_run"] else 5000 #10000
train_config["batches_per_epoch"] = int(8/train_config["batch_size"] * train_config["batches_per_epoch"])
train_config["file_load_fraction"] = 0.5 if general_config["short_training_run"] else 0.3
train_config["num_prefetch_batch"] = 1
train_config["num_workers"] = 2 # will make run nondeterministic
train_config["burn_in_time"] = 150
train_config["input_window_size"] = 500
train_config["rest_start"] = False
train_config["sec_len"] = 200
train_config["start_save_path"] = '../data_processed/NeuronIOstartpoint/'
train_config["ignore_time_from_start"] = 400

## Data, Model, Training Setup

In [10]:
# Preparing Data Loaders

train_files = get_data_files_from_folder(data_config["train_data_dirs"])
valid_files = get_data_files_from_folder(data_config["valid_data_dirs"])
test_files = get_data_files_from_folder(data_config["test_data_dirs"])

train_data_loader = NeuronIO(
    file_paths=train_files,
    batches_per_epoch=train_config["batches_per_epoch"],
    batch_size=train_config["batch_size"],
    input_window_size=train_config["input_window_size"],
    num_workers=train_config["num_workers"],
    num_prefetch_batch=train_config["num_prefetch_batch"],
    file_load_fraction=train_config["file_load_fraction"],
    rest_start = train_config["rest_start"],
    sec_len = train_config["sec_len"],
    save_path = train_config["start_save_path"],
    seed=general_config["seed"],
    ignore_time_from_start = train_config["ignore_time_from_start"],
    device=torch_device,
)

train_evaluator = NeuronioEvaluator(
    test_file=train_files[0],
    burn_in_time=train_config["burn_in_time"],
    input_window_size=train_config["input_window_size"],
    device=torch_device,
)

valid_evaluator = NeuronioEvaluator(
    test_file=valid_files[0],
    burn_in_time=train_config["burn_in_time"],
    input_window_size=train_config["input_window_size"],
    device=torch_device,
)

In [14]:
# Visualize training data
X_viz, (y_spike_viz, y_soma_viz) = next(iter(train_data_loader))
visualize_training_batch(X_viz, y_spike_viz, y_soma_viz, num_viz=8)

NameError: name 'train_data_loader' is not defined

In [9]:
# Initialize the ELM model
if train_config['forget_gate']:
    model = ELMf(**model_config).to(torch_device)
else:
    model = ELM(**model_config).to(torch_device)

# Initialize the loss function, optimizer, and scheduler
criterion= NeuronioLoss()
optimizer = optim.Adam(model.parameters(), lr=train_config["learning_rate"])
scheduler = CosineAnnealingLR(
    optimizer, 
    T_max=train_config["batches_per_epoch"] * train_config["num_epochs"]
)

In [ ]:
# Visualize ELM model
print(model)

## Training

In [11]:
# Initialize the best validation RMSE to a high value
best_valid_rmse = float('inf')
best_model_state_dict = model.state_dict().copy()

# Training loop
train_rmse_hist = []
train_auc_hist = []
valid_rmse_hist = []
valid_auc_hist = []
for epoch in range(train_config["num_epochs"]):
    # Training
    model.train()
    running_loss = 0.0
    pbar = tqdm(
        enumerate(train_data_loader, 0),
        total=train_config["batches_per_epoch"],
        disable=not general_config["verbose"],
    )
    for i, data in pbar:
        inputs, targets = data
        
        # Perform a single training step
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update running loss
        running_loss += loss.item()
        pbar.set_description(f"Epoch {epoch+1} Loss: {running_loss / (i+1):.5f}")
    
    model.eval()
    with torch.no_grad():
        # Evaluate on training data
        train_eval_metrics = train_evaluator.evaluate(model)
        train_rmse = train_eval_metrics["soma_RMSE"]
        train_auc = train_eval_metrics["AUC"]
        train_rmse_hist.append(train_rmse)
        train_auc_hist.append(train_auc)

        # Evaluate on validation data
        valid_eval_metrics = valid_evaluator.evaluate(model)
        valid_rmse = valid_eval_metrics["soma_RMSE"]
        valid_auc = valid_eval_metrics["AUC"]
        valid_rmse_hist.append(valid_rmse)
        valid_auc_hist.append(valid_auc)

    # Copy model state dict if validation RMSE has improved
    if valid_rmse < best_valid_rmse:
        best_valid_rmse = valid_rmse
        best_model_state_dict = model.state_dict().copy()

    # Print statistics
    print(
        f'Epoch: {epoch+1}, '
        f'Train Loss: {running_loss / train_config["batches_per_epoch"]:.5f}, '
        f'Train RMSE: {train_rmse:.5f}, Train AUC: {train_auc:.5f}, '
        f'Valid RMSE: {valid_rmse:.5f}, Valid AUC: {valid_auc:.5f}'
    )

# Free up memory
del train_data_loader
gc.collect()

# Load the best model for evaluation
model.load_state_dict(best_model_state_dict)
model.eval();

Epoch 1 Loss: 0.09125: 100%|█████████████████████████████████████████████████████████| 500/500 [04:34<00:00,  1.82it/s]


Epoch: 1, Train Loss: 0.09125, Train RMSE: 1.81739, Train AUC: 0.71932, Valid RMSE: 1.81899, Valid AUC: 0.72942


Epoch 2 Loss: 0.02498: 100%|█████████████████████████████████████████████████████████| 500/500 [04:03<00:00,  2.06it/s]


Epoch: 2, Train Loss: 0.02498, Train RMSE: 1.54311, Train AUC: 0.95956, Valid RMSE: 1.51724, Valid AUC: 0.96138


Epoch 3 Loss: 0.02058: 100%|█████████████████████████████████████████████████████████| 500/500 [03:53<00:00,  2.14it/s]


Epoch: 3, Train Loss: 0.02058, Train RMSE: 1.46073, Train AUC: 0.96512, Valid RMSE: 1.41557, Valid AUC: 0.96787


Epoch 4 Loss: 0.02021: 100%|█████████████████████████████████████████████████████████| 500/500 [03:52<00:00,  2.15it/s]


Epoch: 4, Train Loss: 0.02021, Train RMSE: 1.43859, Train AUC: 0.96613, Valid RMSE: 1.39075, Valid AUC: 0.96856


Epoch 5 Loss: 0.01987: 100%|█████████████████████████████████████████████████████████| 500/500 [03:55<00:00,  2.12it/s]


Epoch: 5, Train Loss: 0.01987, Train RMSE: 1.43470, Train AUC: 0.96629, Valid RMSE: 1.38508, Valid AUC: 0.96865


## Evaluation

In [ ]:
# Visualize predictions
with torch.no_grad():
    outputs = model.neuronio_eval_forward(X_viz)
    visualize_training_batch(X_viz, y_spike_viz, y_soma_viz, outputs[..., 0], outputs[..., 1], num_viz=8)

In [15]:
# Gather all evaluation metrics
with torch.no_grad():
    if not general_config["short_training_run"]:
        random.seed(general_config["seed"])
        select_train_files = random.choices(train_files, k=10)
        train_predictions = compute_test_predictions_multiple_sim_files(
            neuron=model,
            test_files=select_train_files,
            burn_in_time=train_config["burn_in_time"],
            input_window_size=train_config["input_window_size"],
            device=torch_device,
        )
        train_results = filter_and_extract_core_results(*train_predictions, verbose=False)

        valid_predictions = compute_test_predictions_multiple_sim_files(
            neuron=model,
            test_files=valid_files,
            burn_in_time=train_config["burn_in_time"],
            input_window_size=train_config["input_window_size"],
            device=torch_device,
        )
        valid_results = filter_and_extract_core_results(*valid_predictions, verbose=False)

    test_predictions = compute_test_predictions_multiple_sim_files(
        neuron=model,
        test_files=test_files,
        burn_in_time=train_config["burn_in_time"],
        input_window_size=train_config["input_window_size"],
        device=torch_device,
    )
    test_results = filter_and_extract_core_results(*test_predictions, verbose=False)

eval_results = dict()
if not general_config["short_training_run"]:
    eval_results["train_results"] = train_results
    eval_results["valid_results"] = valid_results
eval_results["test_results"] = test_results

In [16]:
# Show evaluation results
eval_results

{'test_results': {'TP @ 0.0025 FP': np.float64(0.16651319828115407),
  'AUC @ 0.0025 FP': np.float64(0.08920503376304481),
  'TP @ 0.0100 FP': np.float64(0.43784530386740333),
  'AUC @ 0.0100 FP': np.float64(0.2611127302835369),
  'AUC': np.float64(0.9643355718697292),
  'soma_explained_variance_percent': 83.82370825049937,
  'soma_RMSE': np.float64(1.233571460924803),
  'soma_MAE': 0.8693042140735558}}